# Estandarización y Transformación de Datos - Telecom X

## Propósito
En este cuaderno, se estandarizan y transforman los datos del dataset de Telecom X para facilitar el análisis y modelado. Se convierten variables categóricas binarias a valores numéricos (1/0) y se renombran las columnas a términos en español para mejorar la claridad y comunicación.

## Tarea
1. Cargar el dataset limpio desde la URL de la API y aplicar correcciones previas.
2. Convertir variables categóricas binarias (como `Churn`, `customer_Partner`, etc.) a valores 1/0.
3. Renombrar las columnas a nombres descriptivos en español.
4. Verificar la coherencia del dataset tras las transformaciones.
5. Guardar el dataset estandarizado.

In [1]:
# Importación de bibliotecas
import pandas as pd
import requests
import json

# Carga de datos desde la API
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"
response = requests.get(url)
data = response.json()

# Conversión a DataFrame
df = pd.DataFrame(data)

# Desanidar columnas JSON
customer_df = pd.json_normalize(df['customer'])
customer_df.columns = [f"customer_{col}" for col in customer_df.columns]
phone_df = pd.json_normalize(df['phone'])
phone_df.columns = [f"phone_{col}" for col in phone_df.columns]
internet_df = pd.json_normalize(df['internet'])
internet_df.columns = [f"internet_{col}" for col in internet_df.columns]
account_df = pd.json_normalize(df['account'])
account_df.columns = [f"account_{col}" for col in account_df.columns]

# Combinar los datos desanidados
df_clean = pd.concat([df[['customerID', 'Churn']], customer_df, phone_df, internet_df, account_df], axis=1)

# Aplicar correcciones previas
df_clean['Churn'] = df_clean['Churn'].replace('', 'No')
df_clean['account_Charges.Monthly'] = pd.to_numeric(df_clean['account_Charges.Monthly'], errors='coerce')
df_clean['account_Charges.Total'] = pd.to_numeric(df_clean['account_Charges.Total'], errors='coerce')
df_clean['customer_tenure'] = df_clean['customer_tenure'].astype('int64')
df_clean['customer_SeniorCitizen'] = df_clean['customer_SeniorCitizen'].astype('int64')

# Imputar valores nulos en 'account_Charges.Total'
median_total = df_clean['account_Charges.Total'].median()
df_clean['account_Charges.Total'] = df_clean.apply(
    lambda row: row['account_Charges.Monthly'] if pd.isna(row['account_Charges.Total']) and row['customer_tenure'] <= 1
    else row['account_Charges.Total'] if not pd.isna(row['account_Charges.Total'])
    else median_total,
    axis=1
)

# Crear la columna 'Cuentas_Diarias'
df_clean['Cuentas_Diarias'] = df_clean['account_Charges.Monthly'] / 30

# 1. Convertir variables categóricas binarias a 1/0
binary_columns = [
    'Churn', 'customer_Partner', 'customer_Dependents', 'phone_PhoneService',
    'account_PaperlessBilling', 'internet_OnlineSecurity', 'internet_OnlineBackup',
    'internet_DeviceProtection', 'internet_TechSupport', 'internet_StreamingTV',
    'internet_StreamingMovies'
]

for col in binary_columns:
    df_clean[col] = df_clean[col].map({'Yes': 1, 'No': 0, 'No internet service': 0, 'No phone service': 0})

# 2. Renombrar columnas a términos en español
column_mapping = {
    'customerID': 'ID_Cliente',
    'Churn': 'Evasión',
    'customer_gender': 'Género',
    'customer_SeniorCitizen': 'Ciudadano_Senior',
    'customer_Partner': 'Pareja',
    'customer_Dependents': 'Dependientes',
    'customer_tenure': 'Antigüedad',
    'phone_PhoneService': 'Servicio_Telefónico',
    'phone_MultipleLines': 'Líneas_Múltiples',
    'internet_InternetService': 'Servicio_Internet',
    'internet_OnlineSecurity': 'Seguridad_Online',
    'internet_OnlineBackup': 'Respaldo_Online',
    'internet_DeviceProtection': 'Protección_Dispositivo',
    'internet_TechSupport': 'Soporte_Técnico',
    'internet_StreamingTV': 'Streaming_TV',
    'internet_StreamingMovies': 'Streaming_Películas',
    'account_Contract': 'Contrato',
    'account_PaperlessBilling': 'Facturación_Sin_Papel',
    'account_PaymentMethod': 'Método_Pago',
    'account_Charges.Monthly': 'Cargos_Mensuales',
    'account_Charges.Total': 'Cargos_Totales',
    'Cuentas_Diarias': 'Cargos_Diarios'
}

df_clean = df_clean.rename(columns=column_mapping)

# 3. Verificar coherencia
print("Valores nulos tras estandarización:")
print(df_clean.isnull().sum())

print("\nTipos de datos tras estandarización:")
print(df_clean.dtypes)

print("\nPrimeras 5 filas del DataFrame estandarizado:")
print(df_clean.head())

# 4. Verificar valores en columnas binarias
for col in ['Evasión', 'Pareja', 'Dependientes', 'Servicio_Telefónico', 'Facturación_Sin_Papel',
            'Seguridad_Online', 'Respaldo_Online', 'Protección_Dispositivo', 'Soporte_Técnico',
            'Streaming_TV', 'Streaming_Películas']:
    print(f"\nValores únicos en {col}:")
    print(df_clean[col].value_counts())

# 5. Guardar el dataset estandarizado
df_clean.to_csv('TelecomX_Data_standardized.csv', index=False)
print("\nDataset estandarizado guardado como 'TelecomX_Data_standardized.csv'.")

Valores nulos tras estandarización:
ID_Cliente                0
Evasión                   0
Género                    0
Ciudadano_Senior          0
Pareja                    0
Dependientes              0
Antigüedad                0
Servicio_Telefónico       0
Líneas_Múltiples          0
Servicio_Internet         0
Seguridad_Online          0
Respaldo_Online           0
Protección_Dispositivo    0
Soporte_Técnico           0
Streaming_TV              0
Streaming_Películas       0
Contrato                  0
Facturación_Sin_Papel     0
Método_Pago               0
Cargos_Mensuales          0
Cargos_Totales            0
Cargos_Diarios            0
dtype: int64

Tipos de datos tras estandarización:
ID_Cliente                 object
Evasión                     int64
Género                     object
Ciudadano_Senior            int64
Pareja                      int64
Dependientes                int64
Antigüedad                  int64
Servicio_Telefónico         int64
Líneas_Múltiples         

## Descripción de la tarea
En esta sección, se convirtieron las variables categóricas binarias (como `Evasión`, `Pareja`, etc.) a valores numéricos (1/0), tratando `"No internet service"` y `"No phone service"` como 0. Se renombraron las columnas a términos descriptivos en español para mejorar la claridad. Se verificó que no haya valores nulos y que los tipos de datos sean correctos. El dataset estandarizado se guardó como `TelecomX_Data_standardized.csv` para su uso en el análisis exploratorio.